In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
'''
데이터 전처리 및 로드
'''
train_trans = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
#
test_trans = transforms.ToTensor()

train_dataset = torchvision.datasets.CIFAR10(root = "D:/programming/week7/data", download = True, train = True, transform = train_trans)
test_dataset = torchvision.datasets.CIFAR10(root = "D:/programming/week7/data", download = True, train = False, transform = test_trans)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128, shuffle = True)

class_names = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

Files already downloaded and verified
Files already downloaded and verified


num = np.random.randint(0, len(test_loader) + 1, 16)
for i, idx in enumerate(num, 1):
    plt.subplot(4, 4, i)
    img = test_dataset[idx][0]
    label = test_dataset[idx][1]
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img)
    plt.title(f"{class_names[label]}", fontsize = 8)
    plt.axis("off")
plt.show()

In [3]:
class Residual_Block(nn.Module):
    ex = 1
    def __init__(self, in_channels, out_channels, stride = 1):
        super(Residual_Block, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels * self.ex, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels * self.ex)
        )
        self.relu = nn.ReLU()

        if (stride != 1) or (in_channels != out_channels * self.ex):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.ex, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.ex)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        x += self.shortcut(residual)
        x = self.relu(x)
        return x

In [4]:
class Bottleneck(nn.Module):
    ex = 4
    def __init__(self, in_channels, out_channels, stride = 1):
        super(Bottleneck, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels * self.ex, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels * self.ex)
        )
        self.relu = nn.ReLU()

        if (stride != 1) or (in_channels != out_channels * self.ex):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.ex, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.ex)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x += self.shortcut(residual)
        x = self.relu(x)
        return x

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        self.conv2 = self.make_layer(block, 64, num_block[0], 1)
        self.conv3 = self.make_layer(block, 128, num_block[1], 2)
        self.conv4 = self.make_layer(block, 256, num_block[2], 2)
        self.conv5 = self.make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.ex, num_classes)

    def make_layer(self, block, out_channels, num_block, stride):
        strides = [stride] + [1] * (num_block - 1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.ex
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [6]:
def ResNet18():
    return ResNet(Residual_Block, [2, 2, 2, 2])
def ResNet34():
    return ResNet(Residual_Block, [3, 4, 6, 3])
def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])
def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])
def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [7]:
model = ResNet18().cuda()
lr = 0.1
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 0.0001)

num_epochs = 3
image = []
real_label = []
pred_label = []

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    cnt = 0
    size = 0

    for idx, (img, lbl) in enumerate(train_loader):
        img = img.cuda()
        lbl = lbl.cuda()

        optimizer.zero_grad()

        out = model(img)
        loss = criterion(out, lbl)
        loss.backward()
        optimizer.step()

        _, predict = torch.max(out, dim = 1)
        train_loss += loss.item()
        size += lbl.size(0)
        cnt += torch.sum(predict == lbl).item()

    train_acc = cnt / size * 100
    print(f"[Epoch - {epoch + 1}]")
    print(f"Train loss = {train_loss:.6f}, Train acc = {train_acc:.2f}%")

In [9]:
def test(epoch):
    model.eval()
    test_loss = 0
    cnt = 0
    size = 0

    with torch.no_grad():
        for img, lbl in test_loader:
            img = img.cuda()
            lbl = lbl.cuda()

            out = model(img)
            loss = criterion(out, lbl)

            _, predict = torch.max(out, dim = 1)
            test_loss += loss.item()
            size += lbl.size(0)
            cnt += torch.sum(predict == lbl).item()

        if epoch + 1 == num_epochs:
            image.append(img)
            real_label.append(lbl)
            pred_label.append(predict)

        test_acc = cnt / size * 100
        print(f"Test loss = {test_loss:.6f}, Test acc = {test_acc:.2f}%")

In [10]:
for epoch in range(num_epochs):
    train(epoch)
    test(epoch)

[Epoch - 1]
Train loss = 854.178908, Train acc = 26.85%
Test loss = 130.775236, Test acc = 38.60%
[Epoch - 2]
Train loss = 620.930602, Train acc = 41.20%
Test loss = 121.181021, Test acc = 44.59%
[Epoch - 3]
Train loss = 545.562231, Train acc = 49.36%
Test loss = 109.526507, Test acc = 50.43%


In [26]:
real_label[0][15].cpu().item()

8

In [25]:
np.transpose(image[0][15].cpu(), (1, 2, 0)).shape

torch.Size([32, 32, 3])

for i in range(16):
    plt.subplot(4, 4, i + 1)
    image = np.transpose(image[0][i].cpu(), (1, 2, 0))
    plt.imshow(image)
    plt.title(f"real - {class_names[real_label[0][i].cpu().item()]}, pred - {class_names[pred_label[0][i].cpu().item()]}", fontsize = 8)
    plt.axis("off")
plt.show()